# Introduction

At the forefront of AI innovation, a leading company specializing in the development of AI-driven solutions aims to enhance customer support services with their latest endeavor: engineering a text classification system that can automatically categorize customer complaints.

In this project, I am tasked with creating a sophisticated machine learning model that can accurately assign complaints to specific categories such as mortgage, credit card, money transfers, debt collection, and more. This model aims to streamline the process of handling customer complaints, ensuring that issues are directed to the appropriate departments swiftly and efficiently. By leveraging advanced text classification techniques, the goal is to improve customer satisfaction and operational efficiency.

In [1]:
!pip install torchmetrics

In [2]:
#importing required libraries

from collections import Counter
import nltk, json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Import data and labels

In [4]:
# Import data and labels
with open("/kaggle/input/ticket-classification/words.json", 'r') as f1:
    words = json.load(f1)
with open("/kaggle/input/ticket-classification/text.json", 'r') as f2:
    text = json.load(f2)
labels = np.load('/kaggle/input/labels/labels.npy')

# Mapping Words to Indices and Padding Sentences

In [5]:
# Dictionaries to store the word to index mappings and vice versa
word2idx = {o:i for i,o in enumerate(words)}
idx2word = {i:o for i,o in enumerate(words)}

# Looking up the mapping dictionary and assigning the index to the respective words
for i, sentence in enumerate(text):
    text[i] = [word2idx[word] if word in word2idx else 0 for word in sentence]
    
# Defining a function that either shortens sentences or pads sentences with 0 to a fixed length
def pad_input(sentences, seq_len):
    features = np.zeros((len(sentences), seq_len),dtype=int)
    for ii, review in enumerate(sentences):
        if len(review) != 0:
            features[ii, -len(review):] = np.array(review)[:seq_len]
    return features

text = pad_input(text, 50)

# Splitting dataset

In [6]:
# Splitting dataset
train_text, test_text, train_label, test_label = train_test_split(text, labels, test_size=0.2, random_state=42)

train_data = TensorDataset(torch.from_numpy(train_text), torch.from_numpy(train_label).long())
test_data = TensorDataset(torch.from_numpy(test_text), torch.from_numpy(test_label).long())

# Creating Data Loaders

In [7]:
# Set the batch size for loading data
batch_size = 400

# Create a DataLoader for the training data with shuffling enabled
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size)

# Create a DataLoader for the test data with shuffling disabled
test_loader = DataLoader(test_data, shuffle=False, batch_size=batch_size)


# Define the Ticket Classifier Model

In [8]:
# Define the classifier class
class TicketClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, target_size):
        super(TicketClassifier, self).__init__()
        # Embedding layer to convert word indices to dense vectors
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        # Convolutional layer to capture local features in the text
        self.conv = nn.Conv1d(embed_dim, embed_dim, kernel_size=3, stride=1, padding=1)
        # Fully connected layer for classification
        self.fc = nn.Linear(embed_dim, target_size)

    def forward(self, text):
        # Pass the input text through the embedding layer and rearrange dimensions for convolution
        embedded = self.embedding(text).permute(0, 2, 1)
        # Apply convolution and ReLU activation
        conved = F.relu(self.conv(embedded))
        # Average pooling across the sequence length
        conved = conved.mean(dim=2) 
        # Pass the pooled features through the fully connected layer
        return self.fc(conved)

# Define Model Hyperparameters

In [9]:
# Calculate the vocabulary size (including an extra token for padding/unknown words)
vocab_size = len(word2idx) + 1

# Determine the number of unique target categories
target_size = len(np.unique(labels))

# Set the dimensionality of the embedding vectors
embedding_dim = 64


# Initialize Model, Loss Function, and Optimizer

In [10]:
# Create an instance of the TicketClassifier class
model = TicketClassifier(vocab_size, embedding_dim, target_size)

# Set the learning rate for the optimizer
lr = 0.05

# Define the loss function as cross-entropy loss
criterion = nn.CrossEntropyLoss()

# Use the Adam optimizer for training the model
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

# Set the number of training epochs
epochs = 3

# Train the Model

In [11]:
# Set the model to training mode
model.train()

# Loop over the number of epochs
for i in range(epochs):
    running_loss, num_processed = 0, 0

    # Iterate over the training data in batches
    for inputs, labels in train_loader:
        # Zero the gradients for the optimizer
        model.zero_grad()

        # Forward pass: compute the model output
        output = model(inputs)

        # Compute the loss
        loss = criterion(output, labels)

        # Backward pass: compute the gradients
        loss.backward()

        # Update the model parameters
        optimizer.step()

        # Accumulate the loss and the number of processed samples
        running_loss += loss.item()
        num_processed += len(inputs)

    # Print the average loss for the epoch
    print(f"Epoch: {i+1}, Loss: {running_loss/num_processed}")


Epoch: 1, Loss: 0.003781237334012985
Epoch: 2, Loss: 0.001505886197090149
Epoch: 3, Loss: 0.0006394104398787021


# Define Evaluation Metrics

In [12]:
# Import the necessary metric functions
from torchmetrics import Accuracy, Precision, Recall

# Define the accuracy metric for a multiclass classification task
accuracy_metric = Accuracy(task='multiclass', num_classes=5)

# Define the precision metric for each class in a multiclass classification task
precision_metric = Precision(task='multiclass', num_classes=5, average=None)

# Define the recall metric for each class in a multiclass classification task
recall_metric = Recall(task='multiclass', num_classes=5, average=None)

# Evaluate Model on Test Set

In [13]:
# Set the model to evaluation mode
model.eval()

# Initialize an empty list to store predicted labels
predicted = []

# Iterate over the test data in batches
for i, (inputs, labels) in enumerate(test_loader):
    # Forward pass: compute the model output
    output = model(inputs)
    
    # Predict the class with the highest probability
    cat = torch.argmax(output, dim=-1)
    
    # Extend the predicted list with the predicted labels
    predicted.extend(cat.tolist())
    
    # Update evaluation metrics with current batch
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)


In [14]:
# Compute the evaluation metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()

# Print the computed metrics
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)


Accuracy: 0.796999990940094
Precision (per class): [0.704081654548645, 0.6638655662536621, 0.8972973227500916, 0.8297872543334961, 0.9274611473083496]
Recall (per class): [0.71875, 0.8315789699554443, 0.7685185074806213, 0.8125, 0.8523809313774109]


# Inferences

**Accuracy:** The model achieves an accuracy of 79.1% on the test set, indicating its overall effectiveness in correctly predicting class labels.

**Precision (per class):** Precision values vary across classes, with the highest for Class 4 (95.1%) and the lowest for Class 1 (66.1%). This indicates the model's ability to minimize false positives, particularly strong in distinguishing Class 4 instances.

**Recall (per class):** The model shows varied recall rates across classes, ranging from 70.3% to 83.8%. This reflects its capability to capture true positives within each class, with Class 1 and Class 4 showing notably higher recall rates.

These insights collectively highlight the model's strengths in correctly identifying specific categories of data, with a focus on precision for minimizing misclassifications and recall for comprehensive coverage of relevant instances.